# Actividad 3: Creación de Gráficos y Visualizaciones Interactivas para un Dataset

## Configuración de entorno virtual

In [1]:
!python -m venv .venv

In [2]:
!.venv\Scripts\activate

## Instalación de librerías

In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Importación de librerías

In [4]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from __future__ import annotations

from enum import Enum
from pathlib import Path
from itertools import product
from dataclasses import dataclass
from typing import Optional, List, Tuple
from sklearn.impute import SimpleImputer, KNNImputer

## Helper de impresión

In [5]:
class MessageType(str, Enum):
    OK = "OK"
    ERROR = "ERROR"
    WARNING = "WARNING"
    INFO = "INFO"
    DEBUG = "DEBUG"


COLOR_MAP = {
    MessageType.OK: "\033[92m",
    MessageType.ERROR: "\033[91m",
    MessageType.WARNING: "\033[93m",
    MessageType.INFO: "\033[94m",
    MessageType.DEBUG: "\033[95m",
}


def custom_print(type: MessageType, message: str):
    """
    Description
    -----------
    Función que permite imprimir mensajes con colores

    Parameters
    ----------
    type : MessageType
        Tipo de mensaje
    message : str
        Mensaje a imprimir
    """
    color = COLOR_MAP.get(type, "\033[0m")
    print(f"{color}[{type}]\033[0m {message}")

Este código define una función `custom_print` que imprime mensajes en consola con etiquetas de tipo y colores personalizados, facilitando la lectura y clasificación visual de salidas durante la ejecución de programas. Para ello, se utiliza una clase `MessageType` basada en `Enum`, que enumera cinco tipos de mensajes: `"OK"`, `"ERROR"`, `"WARNING"`, `"INFO"` y `"DEBUG"`, cada uno asociado a un color ANSI en el diccionario `COLOR_MAP`. La función recibe un tipo de mensaje y un texto, selecciona el color correspondiente y lo imprime con formato `[TIPO] mensaje`, aplicando el color solo al prefijo.

Esta estructura modular mejora la legibilidad del código, evita errores por escritura manual de cadenas y permite extender fácilmente nuevos tipos de mensajes o estilos. Además, el uso de `Enum` fortalece el tipado estático y la autocompletación en entornos de desarrollo, lo que resulta útil tanto en proyectos técnicos como en recursos pedagógicos institucionales.

## Configuración Global

In [6]:
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
OUTPUT_SCATTER = Path("output/scatter")
OUTPUT_SCATTER.mkdir(parents=True, exist_ok=True)

OUTPUT_BAR = Path("output/bar")
OUTPUT_BAR.mkdir(parents=True, exist_ok=True)

OUTPUT_BOXPLOT = Path("output/boxplot")
OUTPUT_BOXPLOT.mkdir(parents=True, exist_ok=True)

OUTPUT_HEATMAP = Path("output/heatmap")
OUTPUT_HEATMAP.mkdir(parents=True, exist_ok=True)

In [8]:
sns.set_theme(style="whitegrid", context="notebook")

In [9]:
@dataclass
class VizConfig:
    """
    Description
    -----------
    Centraliza los parámetros necesarios para la visualización de datos.
    """
    csv_path: Optional[str] = None
    seaborn_dataset: Optional[str] = None
    has_header: Optional[bool] = True
    headers: Optional[List[str]] = None
    x: Optional[str] = None
    y: Optional[str] = None
    cat: Optional[str] = None
    sample_rows: Optional[int] = None

Este fragmento define una clase de configuración llamada `VizConfig` usando el decorador `@dataclass`, que permite crear objetos con atributos predefinidos de forma concisa y legible. Esta clase está diseñada para centralizar los parámetros necesarios en procesos de visualización de datos, como la ruta del archivo CSV (`csv_path`), el nombre de un dataset de seaborn (`seaborn_dataset`), si el archivo tiene encabezados (`has_header`), y una lista personalizada de nombres de columnas (`headers`).

Además, incluye atributos opcionales para especificar las variables que se usarán en los ejes de los gráficos (`x` y `y`), una variable categórica (`cat`) y un número límite de filas a muestrear (`sample_rows`).


## Carga de datos

In [10]:
def load_dataset(cfg: VizConfig) -> pd.DataFrame:
    """
    Description
    -----------
    Carga un dataset desde un archivo CSV o desde un dataset de Seaborn.
    
    Parameters
    ----------
    cfg : VizConfig
        Configuración para cargar el dataset.
    
    Returns
    -------
    pd.DataFrame
        DataFrame con el dataset cargado.
    """
    custom_print("INFO", "Cargando dataset...")

    if cfg.csv_path:
        if cfg.has_header:
            df = pd.read_csv(cfg.csv_path)
        else:
            df = pd.read_csv(cfg.csv_path, header=None, names=cfg.headers)
    else:
        try:
            name = cfg.seaborn_dataset or "penguins"
            df = sns.load_dataset(name)
        except Exception as e:
            custom_print(MessageType.ERROR, str(e))
            rng = np.random.default_rng(42)

            df = pd.DataFrame({
                "feature_a": rng.normal(0, 1, 200),
                "feature_b": rng.normal(0, 1, 200),
                "category": np.where(rng.random(200) > 0.5, "A", "B")
            })
    
    if cfg.sample_rows and cfg.sample_rows < len(df):
        df = df.sample(n=cfg.sample_rows, random_state=42)
    
    custom_print("OK", f"Dataset cargado con {len(df)} filas")
    return df.reset_index(drop=True)

Esta función `load_dataset` carga un conjunto de datos a partir de una configuración flexible definida por un objeto `VizConfig`. Si se especifica una ruta CSV (`csv_path`), lee el archivo con o sin encabezados según el valor de `has_header`, permitiendo también asignar nombres personalizados a las columnas. Si no se proporciona un CSV, intenta cargar un dataset de seaborn (por defecto, `"penguins"`). En caso de error (por ejemplo, si no hay conexión o el nombre es inválido), genera un DataFrame sintético con dos variables numéricas y una categórica. Además, si se indica un número de filas a muestrear (`sample_rows`), la función toma una muestra aleatoria del conjunto de datos para facilitar análisis exploratorios o visualizaciones rápidas.

## Estandarización de Strings

In [11]:
def standardize_strings(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """
    Description
    -----------
    Estandariza string en un DataFrame al convertirlos a minúsculas, eliminar espacios en blanco al inicio y al final,
    y reemplazar "none" y "nan" con NaN.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con strings a estandarizar.
    cols : List[str]
        Lista de columnas con strings a estandarizar.

    Returns
    -------
    pd.DataFrame
        DataFrame con strings estandarizados.
    """
    custom_print("INFO", "Estandarizando strings...")
    df_out = df.copy()
    for col in cols:
        df_out[col] = df_out[col].astype(str).str.lower().str.strip().replace({"none": np.nan, "nan": np.nan, "?": np.nan})

    custom_print("OK", "Strings estandarizados")
    return df_out

Esta función `standardize_strings` permite estandarizar cadenas de texto en columnas específicas de un DataFrame, lo cual es clave para mejorar la calidad de los datos categóricos antes de realizar análisis, imputaciones o codificaciones. Para cada columna indicada, convierte los valores a minúsculas, elimina espacios en blanco al inicio y al final, y reemplaza las cadenas `"none"` y `"nan"` por valores nulos (`np.nan`), lo que facilita su tratamiento posterior como datos faltantes. Esta limpieza homogénea es especialmente útil cuando se trabaja con datos provenientes de múltiples fuentes o con errores de digitación.

## Imputación de valores faltantes

In [12]:
def impute_missing(df: pd.DataFrame, numeric_strategy: str = "median", categorical_strategy: str = "most_frequent", knn_for_numeric: bool = False, knn_k: int = 5) -> pd.DataFrame:
    """
    Description
    -----------
    Imputa faltantes en:
        - Columnas numéricas: según el parámetro `numeric_strategy`, puede ser:
            - "mean": reemplaza con la media
            - "median": reemplaza con la mediana
            - "KNN": si `knn_for_numeric` es True
        - Columnas categóricas: según el parámetro `categorical_strategy`, puede ser:
            - "most_frequent": reemplaza con la categoría más frecuente (moda)

    Nota: KNNImputer sólo se aplica en numéricas (codifica patrones entre variables).

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con datos
    numeric_strategy : str, optional
        Estrategia para imputar columnas numéricas, by default "median"
    categorical_strategy : str, optional
        Estrategia para imputar columnas categóricas, by default "most_frequent"
    knn_for_numeric : bool, optional
        Si True, se aplica KNNImputer en columnas numéricas, by default False
    knn_k : int, optional
        Valor de k para KNNImputer, by default 5

    Returns
    -------
    pd.DataFrame
        DataFrame con datos imputados
    """
    custom_print("INFO", "Imputando valores faltantes...")
    df_out = df.copy()

    numeric_cols = df_out.select_dtypes(include=[np.number]).columns.to_list()
    categorical_cols = df_out.select_dtypes(exclude=[np.number]).columns.to_list()

    if knn_for_numeric and len(numeric_cols) > 0:
        custom_print("INFO", f"KNNImputer en columnas numéricas (k={knn_k})")
        knn = KNNImputer(n_neighbors=knn_k)
        df_out[numeric_cols] = knn.fit_transform(df_out[numeric_cols])
    else:
        if len(numeric_cols) > 0:
            custom_print("INFO", f"Imputando columnas numéricas con ({numeric_strategy})")
            simple_imputer_numerical = SimpleImputer(strategy=numeric_strategy)
            df_out[numeric_cols] = simple_imputer_numerical.fit_transform(df_out[numeric_cols])

    if len(categorical_cols) > 0:
        custom_print("INFO", f"Imputando columnas categóricas con ({categorical_strategy})")
        simple_imputer_categorical = SimpleImputer(strategy=categorical_strategy)
        df_out[categorical_cols] = simple_imputer_categorical.fit_transform(df_out[categorical_cols])

    custom_print("OK", "Imputación completada")
    return df_out

Esta función `impute_missing` permite imputar valores faltantes en un DataFrame, diferenciando entre columnas numéricas y categóricas, y ofreciendo flexibilidad en la estrategia de imputación. Para las variables numéricas, se puede optar por reemplazar los valores nulos con la media o la mediana, o aplicar un imputador basado en vecinos más cercanos (`KNNImputer`) si se activa el parámetro `knn_for_numeric`. Para las variables categóricas, se utiliza por defecto la moda (valor más frecuente), lo que permite preservar la distribución original. La función identifica automáticamente los tipos de columnas, aplica la estrategia correspondiente, y utiliza `custom_print` para informar cada etapa del proceso, incluyendo la finalización exitosa.

## Eliminación de duplicados

In [13]:
def drop_duplicates(df: pd.DataFrame, subset: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Description
    -----------
    Elimina duplicados; si subset es None, usa todas las columnas

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame a limpiar
    subset : Optional[List[str]], optional
        Columnas a considerar para la detección de duplicados, por defecto None

    Returns
    -------
    pd.DataFrame
        DataFrame limpio de duplicados
    """
    custom_print("INFO", "Eliminando duplicados...")
    before = len(df)
    df_out = df.drop_duplicates(subset=subset)
    after = len(df_out)
    custom_print("OK", f"{before - after} duplicados eliminados")
    return df_out

Esta función `drop_duplicates` elimina filas duplicadas de un DataFrame, permitiendo especificar un subconjunto de columnas (`subset`) para determinar qué registros se consideran repetidos. Si no se indica un subconjunto, se evalúan todas las columnas. Antes y después de aplicar `drop_duplicates`, se calcula la cantidad de filas para informar cuántos duplicados fueron eliminados mediante un mensaje personalizado con `custom_print`, lo que facilita el seguimiento del proceso de limpieza.

## Detección de columnas útiles

In [14]:
def pick_columns(df: pd.DataFrame, x: Optional[str], y: Optional[str], cat: Optional[str]) -> Tuple[str, str, Optional[str]]:
    """
    Description
    -----------
    Selecciona columnas útiles para visualizaciones o análisis exploratorio.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con los datos
    x : Optional[str]
        Columna para el eje x
    y : Optional[str]
        Columna para el eje y
    cat : Optional[str]
        Columna para el eje categórico
    
    Returns
    -------
    Tuple[str, str, Optional[str]]
        Columnas seleccionadas
    """
    custom_print("INFO", "Seleccionando columnas...")

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

    if not x or x not in df.columns:
        x = numeric_cols[0] if numeric_cols else df.columns[0]
    if not y or y not in df.columns:
        if len(numeric_cols) >= 2:
            y = numeric_cols[1]
        else:
            y = df.columns[1]

    if cat and cat not in df.columns:
        cat = None
    if not cat and cat_cols:
        cat = cat_cols[0]

    custom_print("OK", f"Columnas seleccionadas: x={x}, y={y}, cat={cat}")
    return x, y, cat

Esta función `pick_columns` selecciona de forma robusta y automática tres columnas clave de un DataFrame: dos numéricas (`x` y `y`) y una categórica (`cat`), útiles para visualizaciones o análisis exploratorio. Si el usuario no especifica `x` o `y`, o si las columnas indicadas no existen en el DataFrame, la función elige las primeras columnas numéricas disponibles. En caso de que no haya suficientes columnas numéricas, recurre a las primeras columnas disponibles del DataFrame como respaldo.

Para la variable categórica `cat`, si no se proporciona o no es válida, se selecciona la primera columna no numérica disponible. Esta lógica garantiza que siempre se devuelva una combinación válida y coherente de columnas, incluso en contextos con datos incompletos o configuraciones mínimas.

## Gráficos básicos

In [15]:
def plot_scatter(
    df: pd.DataFrame, x: str, y: str, 
    hue: Optional[str] = None,
    style: Optional[str] = None,
    size: Optional[str] = None,
    title_prefix: str = "Scatter",
    annotate_points: bool = False,
    show_grid: bool = True    
) -> None:
    """
    Description
    -----------
    Genera un scatter plot para visualizar la relación entre dos variables
    
    Parameters
    ----------
    df: pd.DataFrame
        DataFrame con los datos
    x: str
        Nombre de la variable x
    y: str
        Nombre de la variable y
    hue: Optional[str] = None
        Nombre de la variable para el color
    style: Optional[str] = None
        Nombre de la variable para el estilo
    size: Optional[str] = None
        Nombre de la variable para el tamaño
    title_prefix: str = "Scatter"
        Prefijo para el titulo
    annotate_points: bool = False
        Indica si se deben anotar los puntos
    show_grid: bool = True
        Indica si se debe mostrar la grilla
    """
    custom_print("INFO", "Generando scatter plot...")

    plt.figure(figsize=(8, 6))
    sns.scatterplot(
        data=df, 
        x=x, 
        y=y, 
        hue=hue, 
        style=style, 
        size=size, 
        alpha=0.8,
        edgecolor="w",
        linewidth=0.5,
    )
    plt.title(f"{title_prefix}: {x} vs {y}", fontsize=14)
    plt.xlabel(x, fontsize=12)
    plt.ylabel(y, fontsize=12)
    
    if show_grid:
        plt.grid(True, linestyle="--", alpha=0.3)
    
    if annotate_points:
        for i in range(len(df)):
            plt.annotate(df[x].iloc[i], df[y].iloc[i], str(i), fontsize=7, alpha=0.6)
    
    plt.tight_layout()
    output_path = OUTPUT_SCATTER / f"scatter_{x}_{y}.png"
    plt.savefig(output_path, dpi=150)
    
    custom_print("OK", f"Scatter plot generado y guardado en {output_path}")
    plt.close()

Esta función `plot_scatter` genera un gráfico de dispersión (scatter plot) a partir de un DataFrame, permitiendo visualizar la relación entre dos variables numéricas (`x` y `y`) y opcionalmente codificar una tercera variable mediante color (`hue`), estilo (`style`) o tamaño (`size`). Está diseñada para ser flexible: permite personalizar el título, mostrar una grilla de fondo y anotar cada punto con su índice, lo cual es útil para análisis exploratorio o presentaciones institucionales.

Internamente, utiliza `seaborn.scatterplot` con parámetros estéticos como transparencia (`alpha`), bordes blancos y líneas delgadas para mejorar la legibilidad. El gráfico se guarda automáticamente en una ruta definida (`OUTPUT_SCATTER`) con un nombre basado en las variables seleccionadas, y se registra el proceso mediante `custom_print`, lo que facilita la trazabilidad en pipelines o notebooks.

In [16]:
def plot_bar(
    df: pd.DataFrame,
    cat: str,
    num: str,
    agg: str = "mean",
    order: Optional[List[str]] = None,
    orient: str = "v",
    show_values: bool = True,
    title_prefix: str = "Bar",
    rotate_xticks: int = 30,
    show_grid: bool = True
) -> None:
    """
    Description
    -----------
    Genera un gráfico de barras con la media de una variable numérica agrupada por una variable categórica.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con los datos.
    cat : str
        Variable categórica para agrupar.
    num : str
        Variable numérica para calcular la media.
    agg : str, optional
        Función de agregación a aplicar, por defecto "mean".
    order : Optional[List[str]], optional
        Orden de las categorías, por defecto None.
    orient : str, optional
        Orientación del gráfico, por defecto "v".
    show_values : bool, optional
        Mostrar valores en el gráfico, por defecto True.
    title_prefix : str, optional
        Prefijo del título, por defecto "Bar".
    rotate_xticks : int, optional
        Rotación de los ticks en el eje x, por defecto 30.
    show_grid : bool, optional
        Mostrar grilla, por defecto True.
    """
    custom_print("INFO", f"Generando bar plot: {agg}({num}) por {cat}")

    grouped = df.groupby(cat, dropna=False)[num].agg(agg).reset_index()

    plt.figure(figsize=(9, 5))
    ax = sns.barplot(
        data=grouped,
        x=cat if orient == "v" else num,
        y=num if orient == "v" else cat,
        order=order,
        orient=orient
    )

    plt.title(f"{title_prefix}: {agg}({num}) por {cat}", fontsize=14)
    if orient == "v":
        plt.xticks(rotation=rotate_xticks, ha="right")
        plt.xlabel(cat, fontsize=12)
        plt.ylabel(f"{agg}({num})", fontsize=12)
    else:
        plt.ylabel(cat, fontsize=12)
        plt.xlabel(f"{agg}({num})", fontsize=12)

    if show_grid:
        plt.grid(axis="y" if orient == "v" else "x", linestyle="--", alpha=0.5)

    if show_values:
        for container in ax.containers:
            ax.bar_label(container, fmt="%.1f", label_type="edge", fontsize=9, padding=3)

    plt.tight_layout()
    output_path = OUTPUT_BAR / f"bar_{agg}_{num}_{cat}.png"
    plt.savefig(output_path, dpi=150)
    custom_print("OK", f"Bar plot generado y guardado en {output_path}")
    plt.close()

Esta función `plot_bar` genera un gráfico de barras a partir de un DataFrame, agregando una variable numérica (`num`) por categorías (`cat`) mediante una función de agregación como `"mean"`, `"sum"` o `"median"`. El gráfico es altamente personalizable: permite definir el orden de las categorías, la orientación (`vertical` u `horizontal`), la rotación de etiquetas, la inclusión de valores sobre las barras y la visualización de una grilla de fondo. Esto la hace especialmente útil para análisis comparativos y presentaciones institucionales. Internamente, la función agrupa los datos, genera el gráfico con `seaborn.barplot`, y ajusta detalles estéticos como títulos, etiquetas y formato de valores. El resultado se guarda automáticamente en una ruta definida (`OUTPUT_BAR`) con un nombre que refleja las variables y la operación aplicada.

## Gráficos avanzados

In [17]:
def plot_boxplot(df: pd.DataFrame, cat: str, num: str) -> None:
    """
    Description
    -----------
    Genera un boxplot para visualizar la distribución de una variable numérica
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame que contiene los datos
    cat : str
        Nombre de la variable categórica
    num : str
        Nombre de la variable numérica
    """
    custom_print("INFO", "Generando boxplot...")

    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x=cat, y=num)
    plt.title(f"Boxplot: {num} vs {cat}")
    plt.tight_layout()
    
    output_path = OUTPUT_BOXPLOT / f"boxplot_{cat}_{num}.png"
    plt.savefig(output_path, dpi=150)

    custom_print("OK", f"Boxplot generado y guardado en {output_path}")
    plt.close()

Esta función `plot_boxplot` genera un gráfico de caja (boxplot) para visualizar la distribución de una variable numérica (`num`) segmentada por una variable categórica (`cat`). Utiliza `seaborn.boxplot` para representar la mediana, los cuartiles y los posibles valores atípicos de cada grupo, lo cual es útil para comparar la dispersión y simetría entre categorías. El gráfico se ajusta automáticamente con `tight_layout` y se guarda como imagen PNG en una ruta predefinida (`OUTPUT_BOXPLOT`), con un nombre que refleja las variables utilizadas.

In [18]:
def plot_heatmap_corr(df: pd.DataFrame) -> None:
    """
    Description
    -----------
    Genera un heatmap de correlación para el DataFrame proporcionado.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame para el cual se generará el heatmap de correlación.
    """
    custom_print("INFO", "Generando heatmap de correlación...")

    num_df = df.select_dtypes(include=[np.number])

    if num_df.shape[1] < 2:
        custom_print(MessageType.ERROR, "Se requieren al menos 2 columnas numéricas para generar un heatmap de correlación")
        return

    corr = num_df.corr(numeric_only=True)
    plt.figure(figsize=(6, 5))
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Mapa de calor de Correlación")
    plt.tight_layout()

    output_path = OUTPUT_HEATMAP / "heatmap_corr.png"
    plt.savefig(output_path, dpi=150)

    custom_print("OK", f"Heatmap de correlación generado y guardado en {output_path}")
    plt.close()

Esta función `plot_heatmap_corr` genera un mapa de calor (heatmap) que visualiza la matriz de correlación entre variables numéricas de un DataFrame. Primero filtra las columnas numéricas y verifica que haya al menos dos para calcular correlaciones significativas. Luego, utiliza `pandas.corr()` para obtener la matriz de correlación y `seaborn.heatmap()` para representarla gráficamente, con anotaciones numéricas (`annot=True`) y una paleta de colores divergente (`coolwarm`) que facilita la interpretación visual de relaciones positivas y negativas.

## Interactividad con Plotly

In [19]:
def px_scatter_interactive(
    df: pd.DataFrame,
    x: str,
    y: str,
    color: Optional[str] = None,
    symbol: Optional[str] = None,
    size: Optional[str] = None,
    hover_data: Optional[List[str]] = None,
    title_prefix: str = "Interactive Scatter",
    template: str = "plotly_white"
) -> None:
    """
    Description
    -----------
    Genera un scatter plot interactivo utilizando Plotly Express.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame que contiene los datos a graficar.
    x : str
        Nombre de la columna que se utilizará como eje x.
    y : str
        Nombre de la columna que se utilizará como eje y.
    color : Optional[str], optional
        Nombre de la columna que se utilizará para colorar los puntos, por defecto None.
    symbol : Optional[str], optional
        Nombre de la columna que se utilizará para simbolizar los puntos, por defecto None.
    size : Optional[str], optional
        Nombre de la columna que se utilizará para el tamaño de los puntos, por defecto None.
    hover_data : Optional[List[str]], optional
        Lista de columnas que se mostrarán al pasar el mouse sobre los puntos, por defecto None.
    title_prefix : str, optional
        Prefijo del título del gráfico, por defecto "Interactive Scatter".
    template : str, optional
        Plantilla de Plotly para el gráfico, por defecto "plotly_white".
    """
    custom_print("INFO", f"Generando scatter plot interactivo: {x} vs {y}")

    fig = px.scatter(
        df,
        x=x,
        y=y,
        color=color,
        symbol=symbol,
        size=size,
        hover_data=hover_data,
        title=f"{title_prefix}: {x} vs {y}",
        template=template,
        opacity=0.7
    )

    fig.update_layout(
        title_font_size=18,
        legend_title_text=color if color else "Legend",
        margin=dict(l=40, r=40, t=60, b=40),
        height=500
    )

    fig.update_traces(marker=dict(line=dict(width=0.5, color="DarkSlateGrey")))

    output_path = OUTPUT_SCATTER / f"scatter_interactive_{x}_{y}.html"
    fig.write_html(str(output_path))

    custom_print("OK", f"Scatter plot interactivo generado y guardado en {output_path}")

Esta función `px_scatter_interactive` genera un gráfico de dispersión interactivo utilizando Plotly Express, ideal para explorar relaciones entre dos variables numéricas (`x` y `y`) con opciones visuales adicionales como color (`color`), símbolo (`symbol`) y tamaño (`size`). También permite incluir información contextual al pasar el cursor (`hover_data`), personalizar el título (`title_prefix`) y aplicar una plantilla estética (`template`). El gráfico se configura con transparencia (`opacity=0.7`), bordes sutiles en los puntos y márgenes ajustados para una presentación limpia.

In [20]:
def px_bar_interactive(
    df: pd.DataFrame,
    cat: str,
    num: str,
    agg: str = "mean",
    color: Optional[str] = None,
    hover_data: Optional[List[str]] = None,
    title_prefix: str = "Interactive Bar",
    template: str = "plotly_white",
    orientation: str = "v",
    show_labels: bool = True
) -> None:
    """
    Description
    -----------
    Genera un gráfico de barras interactivo con Plotly Express.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con los datos.
    cat : str
        Columna categórica para el eje x.
    num : str
        Columna numérica para el eje y.
    agg : str, optional
        Función de agregación (default: "mean").
    color : str, optional
        Columna para el color (default: None).
    hover_data : list, optional
        Columnas adicionales para mostrar en el hover (default: None).
    title_prefix : str, optional
        Prefijo para el título (default: "Interactive Bar").
    template : str, optional
        Template para el gráfico (default: "plotly_white").
    orientation : str, optional
        Orientación del gráfico (default: "v").
    show_labels : bool, optional
        Mostrar etiquetas en el gráfico (default: True).
    """
    custom_print("INFO", f"Generando bar plot interactivo: {agg}({num}) por {cat}")

    grouped = df.groupby(cat, dropna=False)[num].agg(agg).reset_index()

    fig = px.bar(
        grouped,
        x=cat if orientation == "v" else num,
        y=num if orientation == "v" else cat,
        color=color or cat,
        hover_data=hover_data,
        orientation=orientation,
        title=f"{title_prefix}: {agg}({num}) por {cat}",
        template=template
    )

    fig.update_layout(
        title_font_size=18,
        xaxis_title=cat if orientation == "v" else f"{agg}({num})",
        yaxis_title=f"{agg}({num})" if orientation == "v" else cat,
        margin=dict(l=40, r=40, t=60, b=40),
        height=500
    )

    if show_labels:
        fig.update_traces(text=grouped[num].round(1), textposition="outside")

    output_path = OUTPUT_BAR / f"bar_interactive_{agg}_{num}_{cat}.html"
    fig.write_html(str(output_path))

    custom_print("OK", f"Bar plot interactivo generado y guardado en {output_path}")

Esta función `px_bar_interactive` genera un gráfico de barras interactivo con Plotly Express, ideal para visualizar agregaciones de una variable numérica (`num`) agrupada por una variable categórica (`cat`). Permite especificar la función de agregación (`agg`, como `"mean"` o `"sum"`), personalizar el color, orientación (`vertical` u `horizontal`), plantilla visual (`template`) y mostrar etiquetas numéricas sobre las barras (`show_labels`). También admite datos adicionales en el `hover` para enriquecer la exploración interactiva.

In [21]:
def px_box_interactive(
    df: pd.DataFrame,
    cat: str,
    num: str,
    color: Optional[str] = None,
    points: str = "suspectedoutliers",  # opciones: "all", "outliers", "suspectedoutliers", False
    hover_data: Optional[List[str]] = None,
    title_prefix: str = "Interactive Boxplot",
    template: str = "plotly_white",
    orientation: str = "v",
    show_mean_line: bool = True
) -> None:
    """
    Description
    -----------
    Genera un boxplot interactivo con Plotly Express.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con los datos.
    cat : str
        Variable categórica para el eje x (vertical) o y (horizontal).
    num : str
        Variable numérica para el eje y (vertical) o x (horizontal).
    color : Optional[str], optional
        Variable categórica para colorear los boxplots, por defecto None.
    points : str, optional
        Indica qué puntos mostrar en el boxplot, por defecto "suspectedoutliers".
    hover_data : Optional[List[str]], optional
        Columnas adicionales para mostrar en el hover, por defecto None.
    title_prefix : str, optional
        Prefijo para el título del boxplot, por defecto "Interactive Boxplot".
    template : str, optional
        Plantilla para el boxplot, por defecto "plotly_white".
    orientation : str, optional
        Orientación del boxplot, por defecto "v" (vertical).
    show_mean_line : bool, optional
        Indica si mostrar la línea de la media, por defecto True.
    """
    custom_print("INFO", f"Generando boxplot interactivo: {num} vs {cat}")

    fig = px.box(
        df,
        x=cat if orientation == "v" else num,
        y=num if orientation == "v" else cat,
        color=color or cat,
        points=points,
        hover_data=hover_data,
        title=f"{title_prefix}: {num} vs {cat}",
        template=template,
        orientation=orientation
    )

    fig.update_layout(
        title_font_size=18,
        xaxis_title=cat if orientation == "v" else num,
        yaxis_title=num if orientation == "v" else cat,
        margin=dict(l=40, r=40, t=60, b=40),
        height=500
    )

    if show_mean_line:
        fig.add_hline(
            y=df[num].mean(),
            line_dash="dot",
            line_color="red",
            annotation_text=f"Media: {df[num].mean():.2f}",
            annotation_position="top left"
        )

    output_path = OUTPUT_BOXPLOT / f"boxplot_interactive_{cat}_{num}.html"
    fig.write_html(str(output_path))

    custom_print("OK", f"Boxplot interactivo generado y guardado en {output_path}")

Esta función `px_box_interactive` genera un boxplot interactivo con Plotly Express para visualizar la distribución de una variable numérica (`num`) segmentada por una variable categórica (`cat`). Permite personalizar el color de las cajas, el tipo de puntos mostrados (`points`), la orientación del gráfico (`vertical` u `horizontal`), y el estilo visual mediante plantillas (`template`). También admite datos adicionales en el `hover` y puede incluir una línea de referencia con la media de la variable numérica (`show_mean_line`), lo cual es útil para análisis comparativos o pedagógicos.

In [22]:
def px_heatmap_corr_interactive(df: pd.DataFrame) -> None:
    """
    Description
    -----------
    Genera un heatmap interactivo de correlación para un DataFrame.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame para el cual se generará el heatmap de correlación.
    
    Returns
    -------
    None
    """
    custom_print("INFO", "Generando heatmap de correlación interactivo...")
    
    num_df = df.select_dtypes(include=[np.number])
    if num_df.shape[1] < 2:
        custom_print(MessageType.ERROR, "Se requieren al menos 2 columnas numéricas para generar el heatmap")
        return

    corr = num_df.corr(numeric_only=True)
    fig = go.Figure(data=go.Heatmap(
        z=corr.values,
        x=corr.columns,
        y=corr.columns,
        zmin=-1,
        zmax=1,
        colorscale="RdBu",
        showscale=True
    ))
    fig.update_layout(title="Interactive Heatmap de Correlación")
    output_path = OUTPUT_HEATMAP / "heatmap_corr_interactive.html"
    fig.write_html(output_path)
    
    custom_print("OK", f"Heatmap de correlación interactivo generado y guardado en {output_path}")

Esta función `px_heatmap_corr_interactive` genera un mapa de calor interactivo que visualiza la matriz de correlación entre variables numéricas de un DataFrame, utilizando Plotly para facilitar la exploración dinámica. Primero filtra las columnas numéricas y verifica que haya al menos dos para calcular correlaciones significativas. Luego, calcula la matriz de correlación con `pandas.corr()` y la representa con `go.Heatmap`, aplicando una escala de color divergente (`RdBu`) que resalta relaciones positivas y negativas entre variables. El gráfico se configura con límites de color entre -1 y 1, etiquetas en los ejes y una barra de escala para facilitar la interpretación. Se guarda como archivo HTML en una ruta predefinida (`OUTPUT_HEATMAP`), lo que permite su visualización en navegadores.

## Compilar gráficos en archivo json

In [23]:
IMAGE_EXTENSIONS = [".png", ".jpg", ".jpeg", ".gif", ".webp"]
HTML_EXTENSIONS = [".html", ".htm"]
FOLDER_META = {
    "scatter": {
        "title": "Gráficos de dispersión (scatter)",
        "subtitle": "Resultados guardados en la carpeta output/scatter",
    },
    "boxplot": {
        "title": "Gráficos de caja (boxplot)",
        "subtitle": "Resultados guardados en la carpeta output/boxplot",
    },
    "bar": {
        "title": "Gráficos de barras (bar)",
        "subtitle": "Resultados guardados en la carpeta output/bar",
    },
    "heatmap": {
        "title": "Mapas de calor (heatmap)",
        "subtitle": "Resultados guardados en la carpeta output/heatmap",
    },
}

In [24]:
def guess_type(path: Path) -> str | None:
    """
    Description
    -----------
    Devuelve el tipo de archivo según su extensión
    """
    ext = path.suffix.lower()
    if ext in IMAGE_EXTENSIONS:
        return "image"
    if ext in HTML_EXTENSIONS:
        return "html"
    return None

Esta función `guess_type` recibe un objeto `Path` y devuelve una cadena que representa el tipo de archivo según su extensión, o `None` si no se reconoce. Extrae la extensión del archivo (`path.suffix`) en minúsculas y la compara con dos listas predefinidas: `IMAGE_EXTENSIONS` y `HTML_EXTENSIONS`. Si la extensión coincide con alguna de estas, retorna `"image"` o `"html"` respectivamente; de lo contrario, retorna `None`.

In [25]:
def prettify_label(filename: str) -> str:
    """
    Description
    -----------
    Regresa el nombre del archivo sin extension de manera legible
    """
    name = Path(filename).stem
    name = name.replace("_", " ").replace("-", " ")
    return name.title()

La función `prettify_label` toma un nombre de archivo como cadena (`filename`) y devuelve una versión más legible y presentable, ideal para títulos o etiquetas en visualizaciones. Primero extrae el nombre base del archivo sin la extensión (`stem`), luego reemplaza guiones bajos (`_`) y guiones medios (`-`) por espacios, y finalmente convierte el resultado a formato título (`title()`), capitalizando la primera letra de cada palabra.

Por ejemplo, un archivo llamado `"boxplot_interactive_total_ingresos.html"` se transformaría en `"Boxplot Interactive Total Ingresos"`. Esta función es especialmente útil para generar títulos automáticos a partir de nombres de archivo técnicos, facilitando la presentación de recursos en informes o dashboards.

In [26]:
def build_manifest(base_dir: Path = OUTPUT_DIR, folders: List[str] | None = None) -> Dict:
    """
    Description
    -----------
    Construye un manifiesto de los archivos en el directorio base

    Parameters
    ----------
    base_dir: Path
        Directorio base
    folders: List[str] | None
        Lista de carpetas
    
    Returns
    -------
    Dict
        Manifiesto de los archivos
    """
    manifest: Dict[str, Dict] = {}

    if folders is None:
        folders = [p.name for p in base_dir.iterdir() if p.is_dir()]

    for folder in folders:
        folder_path = base_dir / folder
        if not folder_path.exists():
            continue

        items: List[Dict] = []

        for fname in sorted(os.listdir(folder_path)):
            fpath = folder_path / fname
            if not fpath.is_file():
                continue

            ftype = guess_type(fpath)
            if ftype is None:
                continue

            items.append({
                "type": ftype,
                "file": fname,
                "label": prettify_label(fname),
            })
        
        meta = FOLDER_META.get(folder, {})
        title = meta.get("title", f"Carpeta: {folder}")
        subtitle = meta.get("subtitle", f"Resultados guardados en la carpeta output/{folder}")

        manifest[folder] = {
            "title": title,
            "subtitle": subtitle,
            "items": items,
        }

    return manifest

Esta función `build_manifest` construye un diccionario estructurado (manifest) que resume el contenido de carpetas dentro de un directorio base (`base_dir`, por defecto `OUTPUT_DIR`). Si no se especifican carpetas, recorre automáticamente todas las subcarpetas del directorio base. Para cada carpeta válida, inspecciona sus archivos, filtra aquellos que son reconocibles por tipo (`image` o `html`, según `guess_type`), y genera una lista de ítems con su tipo, nombre de archivo y una etiqueta legible (`prettify_label`).

Además, incorpora metadatos opcionales desde un diccionario externo `FOLDER_META`, como títulos y subtítulos personalizados para cada carpeta. El resultado es un diccionario anidado que organiza los recursos por carpeta, ideal para alimentar interfaces web, catálogos interactivos o dashboards institucionales. Esta función es especialmente útil para automatizar la documentación y navegación de resultados visuales generados por scripts o notebooks.

In [27]:
def save_manifest(manifest: Dict, base_dir: Path = OUTPUT_DIR, filename: str = "plots.json") -> Path:
    """
    Description
    -----------
    Guarda el manifest en un archivo JSON.
    
    Parameters
    ----------
    manifest : Dict
        Diccionario con el manifest.
    base_dir : Path, optional
        Directorio base donde se guardara el manifest, by default OUTPUT_DIR
    filename : str, optional
        Nombre del archivo, by default "plots.json"
    
    Returns
    -------
    Path
        Path al archivo guardado
    """
    out_path = base_dir / filename
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with out_path.open("w", encoding="utf-8") as f:
        json.dump(manifest, f, ensure_ascii=False, indent=4)
    
    custom_print("OK", f"Manifest guardado en {out_path.resolve()}")
    return out_path

La función `save_manifest` guarda un diccionario `manifest` como archivo JSON en un directorio base (`base_dir`), con un nombre de archivo configurable (por defecto `"plots.json"`). Primero construye la ruta de salida (`out_path`) y se asegura de que el directorio exista, creando carpetas intermedias si es necesario. Luego escribe el contenido del manifiesto en formato JSON con codificación UTF-8, sin escapar caracteres Unicode (`ensure_ascii=False`) y con indentación para facilitar su lectura.

In [28]:
def compile_plots():
    """
    Description
    -----------
    Genera un manifest con los archivos de los plots generados
    """
    custom_print("INFO", "Generando Manifest")
    folders = ["scatter", "boxplot", "bar", "heatmap"]

    manifest = build_manifest(OUTPUT_DIR, folders=folders)
    save_manifest(manifest) 

La función `compile_plots` automatiza la generación de un manifiesto estructurado que resume los recursos visuales almacenados en subcarpetas específicas del directorio de salida (`OUTPUT_DIR`). Primero define una lista de carpetas relevantes (`scatter`, `boxplot`, `bar`, `heatmap`), luego utiliza `build_manifest` para recorrerlas y construir un diccionario con metadatos sobre los archivos encontrados (tipo, nombre, etiqueta legible). Finalmente, guarda este manifiesto como un archivo JSON mediante `save_manifest`, permitiendo su uso posterior en dashboards, catálogos o interfaces web.

## Pipeline principal

In [29]:
def run_pipeline(cfg: VizConfig) -> None:
    """
    Description
    -----------
    Ejecuta un flujo automatizado de análisis y visualización de datos a partir de una configuración.
    
    Parameters
    ----------
    cfg : VizConfig
        Configuración con los parámetros para el pipeline.
    """
    custom_print("INFO", "Iniciando pipeline...")

    original_df = load_dataset(cfg)

    str_cols = original_df.select_dtypes(exclude=[np.number]).columns.tolist()
    standardized_df = standardize_strings(original_df, str_cols)

    imputed_df = impute_missing(standardized_df)
    
    final_df = drop_duplicates(imputed_df)

    numeric_cols = final_df.select_dtypes(include=[np.number]).columns.tolist()
    combinations = [(x, y) for x, y in product(numeric_cols, repeat=2) if x != y]

    for x, y in combinations:
        cfg.x = x
        cfg.y = y
        
        x, y, cat = pick_columns(final_df, cfg.x, cfg.y, cfg.cat)

        plot_scatter(final_df, x, y, hue=cat)
        
        if (cat):
            plot_bar(final_df, cat, y, agg="mean")
            plot_boxplot(final_df, cat, y)
        
        px_scatter_interactive(final_df, x, y, color=cat)
        
        if (cat):
            px_bar_interactive(final_df, cat, y, agg="mean")
            px_box_interactive(final_df, cat, y)
    
    plot_heatmap_corr(final_df)
    px_heatmap_corr_interactive(final_df)

    custom_print("OK", f"Columnas elegidas: x = {x}, y = {y}, cat = {cat}")

    compile_plots()
    
    custom_print("INFO", "Pipeline completado")

La función `run_pipeline` ejecuta un flujo automatizado de análisis y visualización de datos a partir de una configuración (`cfg`). Comienza cargando el dataset, estandarizando las variables categóricas, imputando valores faltantes y eliminando duplicados. Luego identifica todas las combinaciones posibles de pares de variables numéricas para generar visualizaciones comparativas. Para cada par `(x, y)`, selecciona una variable categórica (`cat`) si está disponible y genera gráficos estáticos (scatter, bar, boxplot) e interactivos (con Plotly) que permiten explorar relaciones y distribuciones. Al final del proceso, se generan mapas de calor de correlación (estático e interactivo) y se compila un manifiesto con metadatos de todos los gráficos exportados.

## Ejecución principal

In [30]:
if __name__ == "__main__":
    start_time = time.time()

    cfg = VizConfig(
        csv_path="data/adult.data",
        # seaborn_dataset="penguins",
        sample_rows=None,
        has_header=False,
        headers=[
            "age", "workclass", "fnlwgt", "education", 
            "education_num", "marital_status", "occupation", 
            "relationship", "race", "sex", "capital_gain", 
            "capital_loss", "hours_per_week", "native_country", "income"
        ]
    )

    run_pipeline(cfg)
    
    end_time = time.time()
    custom_print("OK", f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")

[INFO] Iniciando pipeline...
[INFO] Cargando dataset...
[OK] Dataset cargado con 32561 filas
[INFO] Estandarizando strings...
[OK] Strings estandarizados
[INFO] Imputando valores faltantes...
[INFO] Imputando columnas numéricas con (median)
[INFO] Imputando columnas categóricas con (most_frequent)
[OK] Imputación completada
[INFO] Eliminando duplicados...
[OK] 24 duplicados eliminados
[INFO] Seleccionando columnas...
[OK] Columnas seleccionadas: x=age, y=fnlwgt, cat=workclass
[INFO] Generando scatter plot...
[OK] Scatter plot generado y guardado en output\scatter\scatter_age_fnlwgt.png
[INFO] Generando bar plot: mean(fnlwgt) por workclass
[OK] Bar plot generado y guardado en output\bar\bar_mean_fnlwgt_workclass.png
[INFO] Generando boxplot...
[OK] Boxplot generado y guardado en output\boxplot\boxplot_workclass_fnlwgt.png
[INFO] Generando scatter plot interactivo: age vs fnlwgt
[OK] Scatter plot interactivo generado y guardado en output\scatter\scatter_interactive_age_fnlwgt.html
[INFO]